Player 1 starting position: 6
Player 2 starting position: 9

In [1]:
class DeterministicDie:
    def __init__(self):
        self.k = -3
    
    def __iter__(self):
        return self
    
    def __next__(self):
        self.k = self.k+3
        return 3+self.k % 100+(self.k+1) % 100+(self.k+2) % 100
    
    @property
    def nrolls(self):
        return 3+self.k

In [2]:
x = 6 - 1
y = 9 - 1
sx = 0
sy = 0

die = DeterministicDie()
k = 0
while sy < 1000:
    k += 1
    x = (x + next(die)) % 10
    sx += (x+1)
    if sx>=1000:
        break
    y = (y + next(die)) % 10
    sy += (y+1)
    

print(sx,sy,die.nrolls)

print(min(sx,sy)*die.nrolls)

1006 921 1005
925605


In [3]:
from functools import lru_cache

In [4]:
rolls = list(zip([3, 4, 5, 6, 7, 8, 9], [1, 3, 6, 7, 6, 3, 1]))

@lru_cache
def roll(x, y, sx, sy, turn, swin):
    # x,y: current positions
    # sx, sy: current scores
    # turn: bool - is it p1's turn?
    # returns a tuple (nwx, nwy) counting in how many sub-universes of this p1 and p2 win respectively
    if sx >= swin:
        return 1, 0
    if sy >= swin:
        return 0, 1
    
    nwx = 0
    nwy = 0
    
    for d, mult in rolls:
        nx, ny, nsx, nsy = x, y, sx, sy
        if turn:
            nx = (x + d) % 10
            nsx = sx + nx + 1
        else:
            ny = (y + d) % 10
            nsy = sy + ny + 1
        sub_wins = roll(nx, ny, nsx, nsy, not turn, swin)
        nwx += mult * sub_wins[0]
        nwy += mult * sub_wins[1]
    
    return nwx, nwy

def dirac(x0, y0, swin=21):
    roll.cache_clear()
    return roll(x0-1, y0-1, 0, 0, True, swin)

In [5]:
%%time

xw, yw = dirac(6, 9)

CPU times: user 1min 1s, sys: 51.2 ms, total: 1min 1s
Wall time: 1min 1s


In [6]:
print(xw, yw)
print(max(xw, yw))

486638407378784 413013330504401
486638407378784


### Note on part 1
The (zero-indexed) $k$-th single roll of the deterministic dice is $s_k = 1 + (k \mod 100)$.

The $k$-th composite roll is effectively $\displaystyle r_k = \sum_{j=3k}^{3k+2} s_k \mod 10 = (9k+6) \mod 10 = (6-k) \mod 10$. Each roll is one less than the previous one, starting at $6$.

From now on all numbers are taken to be modulo 10, except the scores. We rescale the positions of the players by one, so that they respect modulo 10 arithmetic. The score increases by one extra point on each turn

The first player gets the even increments, moving with velocity $v_k = 8-2k$, while the second player gets $w_k = 7-2k$.
After $k$ turns the players have position $x_k = x_0+7k-k^2$, $y_k = y_0+6k-k^2$.

In [7]:
x0 = 5
y0 = 8

x = lambda k: (x0+7*k-k**2)%10
y = lambda k: (y0+6*k-k**2)%10

In [8]:
k = 0
sx = 0
sy = 0

while sy<1000:
    k += 1
    sx += 1+x(k)
    if sx >= 1000:
        break
    sy += 1+y(k)


nrolls = k*6 if sx<sy else k*6-3

print(sx,sy,nrolls)

print(min(sx,sy)*nrolls)

1006 921 1005
925605
